In [ ]:
# Every time you want to refresh your app,
# restart your kernel and rerun these TWO cells
from jupyter_dash.comms import _send_jupyter_config_comm_request

_send_jupyter_config_comm_request()

In [ ]:
# Import `app` object from `display.py` module
from display import app
from jupyter_dash import JupyterDash  # noQA F401

JupyterDash.infer_jupyter_proxy_config()

# Start app server
app.run_server(host="0.0.0.0", mode="external")

In [ ]:
from database import MongoRepository
from pymongo.collection import Collection

repo = MongoRepository()

# Is `MongoRepository.collection` correct type?
assert isinstance(repo.collection, Collection)

# Is repo connected to correct collection?
collection_name = repo.collection.name
assert collection_name == "ds-applicants"

print("repo collection:", collection_name)

In [ ]:
import pandas as pd
from database import MongoRepository

repo = MongoRepository()

# Does `MongoRepository.get_nationality_value_counts` return DataFrame?
df = repo.get_nationality_value_counts(normalize=False)
assert isinstance(df, pd.DataFrame)

# Does DataFrame have correct columns?
cols = sorted(df.columns.tolist())
assert cols == ["count", "country_iso2", "country_iso3", "country_name"]
df.head()

In [ ]:
from business import GraphBuilder
from plotly.graph_objects import Figure

gb = GraphBuilder()

# Does `GraphBuilder.build_nat_choropleth` return a Figure?
fig = gb.build_nat_choropleth()
assert isinstance(fig, Figure)
fig.show()

In [ ]:
import pandas as pd
from database import MongoRepository

repo = MongoRepository()
# Does `MongoRepository.get_ages` return a Series?
ages = repo.get_ages()
assert isinstance(ages, pd.Series)
ages.head()

In [ ]:
from business import GraphBuilder
from plotly.graph_objects import Figure

gb = GraphBuilder()

# Does `GraphBuilder.build_nat_choropleth` return a Figure?
fig = gb.build_age_hist()
assert isinstance(fig, Figure)
fig.show()

In [ ]:
import pandas as pd
from database import MongoRepository

# Test  method
repo = MongoRepository()

# Does `MongoRepository.get_ed_value_counts` return a Series?
degrees = repo.get_ed_value_counts(normalize=False)
assert isinstance(degrees, pd.Series)

# Is Series index ordered correctly?
assert degrees.index.tolist() == [
    "High School or Baccalaureate",
    "Some College (1-3 years)",
    "Bachelor's degree",
    "Master's degree",
    "Doctorate (e.g. PhD)",
]

degrees

In [ ]:
from business import GraphBuilder
from plotly.graph_objects import Figure

gb = GraphBuilder()

# Does `GraphBuilder.build_ed_bar` return a Figure?
fig = gb.build_ed_bar()
assert isinstance(fig, Figure)
fig.show()

### EXPERIMENT

from business import StatsBuilder
from database import MongoRepository

sb = StatsBuilder()

# Is `StatsBuilder.repo` the correct data type?
assert isinstance(sb.repo, MongoRepository)
sb.repo.collection.name

In [ ]:
from business import StatsBuilder

# Does `StatsBuilder.calculate_n_obs` return an int?
n_obs = sb.calculate_n_obs(effect_size=0.2)
assert isinstance(n_obs, int)

# Does `StatsBuilder.calculate_n_obs` return correct number?
assert n_obs == 394
print("# observations for effect size of 0.2:", n_obs)

In [ ]:
import pandas as pd
import wqet_grader
from database import MongoRepository
from teaching_tools.ab_test.reset import Reset

# Reset database, just in case
r = Reset()
r.reset_database()

# Initialize grader
wqet_grader.init("Project 7 Assessment")


# Instantiate `MongoRepository`
repo = MongoRepository()

# Does `MongoRepository.get_no_quiz_per_day` return a Series?
no_quiz = repo.get_no_quiz_per_day()
assert isinstance(no_quiz, pd.Series)

# Does `MongoRepository.get_no_quiz_per_day` return correct value?
assert no_quiz.shape == (30,)

print("no_quiz shape:", no_quiz.shape)
print(no_quiz.head())

# Submit `no_quiz` to grader
wqet_grader.grade("Project 7 Assessment", "Task 7.4.15", no_quiz)


In [ ]:
from business import StatsBuilder

sb = StatsBuilder()

# Does `StatsBuilder.calculate_cdf_pct` return a float?
pct = sb.calculate_cdf_pct(n_obs=394, days=12)
assert isinstance(pct, float)

# Does `StatsBuilder.calculate_cdf_pct` return correct value
assert pct > 95
assert pct <= 100

print(f"Probability: {pct}%")

### RESULTS

from business import StatsBuilder
from database import MongoRepository
from teaching_tools.ab_test.experiment import Experiment

mr = MongoRepository()
exp = Experiment(repo=mr)
sb = StatsBuilder()
exp.reset_experiment()

# Does `StatsBuilder.run_experiment` add documents to database?
docs_before_exp = mr.collection.count_documents({})
sb.run_experiment(days=1)
docs_after_exp = mr.collection.count_documents({})
assert docs_after_exp > docs_before_exp

exp.reset_experiment()
print("Documents added to database:", docs_after_exp - docs_before_exp)

In [ ]:
import pandas as pd
from business import StatsBuilder
from database import MongoRepository

sb = StatsBuilder()
mr = MongoRepository()

# Does `MongoRepository.get_contingency_table` return a DataFrame?
sb.run_experiment(days=1)
contingency_tab = mr.get_contingency_table()
assert isinstance(contingency_tab, pd.DataFrame)

# Does `MongoRepository.get_contingency_table` return right shape?
assert contingency_tab.shape == (2, 2)
contingency_tab

In [ ]:
from business import GraphBuilder, StatsBuilder
from plotly.graph_objects import Figure

gb = GraphBuilder()
sb = StatsBuilder()

# Does `GraphBuilder.build_contingency_bar` return a Figure?
sb.run_experiment(days=1)
fig = gb.build_contingency_bar()
assert isinstance(fig, Figure)
fig.show()

In [ ]:
from business import StatsBuilder
from statsmodels.stats.contingency_tables import _Bunch

sb = StatsBuilder()

# Does `StatsBuilder.run_chi_square` return a Bunch?
sb.run_experiment(days=1)
result = sb.run_chi_square()
assert isinstance(result, _Bunch)

# Is Bunch p-value correct?
p_val = result.pvalue
assert p_val > 0.05

print("Experiment p-value:", p_val)